In [1]:
import gym 
from IPython.display import clear_output
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete 
import numpy as np
import random
import os
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy



In [2]:
Discrete(3).sample()

1

In [3]:
Box(0,5,shape=(3,3)).sample()
Box(low=np.array([0]),high=np.array([100]))

/home/vedant/miniconda3/envs/tensorflow/lib/python3.7/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Box(0.0, 100.0, (1,), float32)

In [4]:
Tuple((Discrete(3),Box(0,5,shape=(3,3)))).sample()

(2,
 array([[3.5941195, 2.9945774, 1.8343985],
        [4.752422 , 4.0425224, 2.0007653],
        [1.6715657, 2.769828 , 3.343826 ]], dtype=float32))

In [5]:
Dict({'h':Discrete(3),'j':Box(0,5,shape=(3,3))}).sample()

OrderedDict([('h', 1),
             ('j', array([[3.4568937 , 1.2811489 , 0.6338226 ],
                     [0.66069067, 1.7142467 , 1.7369864 ],
                     [0.61429626, 2.8112361 , 4.3401313 ]], dtype=float32))])

In [6]:
MultiBinary(4).sample()

array([0, 0, 1, 1], dtype=int8)

In [7]:
MultiDiscrete([5,2,2,6]).sample()

array([4, 1, 1, 5])

In [8]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3)
        self.observation_space = Box(low=np.array([0]), high=np.array([100]))
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60
        
    def step(self, action):
        self.state += action -1 
        self.shower_length -= 1 
        
        if 37<=self.state <=39: 
            reward =1 
        else: 
            reward = -1 
        
        if self.shower_length <= 0: 
            done = True
        else:
            done = False
        
        info = {}

        return self.state, reward, done, info

    def render(self):
#         clear_output(wait=True)
#         print(f'Current temp : {self.state}')
        pass
    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60 
        return self.state



In [9]:
env=ShowerEnv()

In [10]:
env.observation_space.sample()

array([79.28478], dtype=float32)

In [11]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()



Episode:1 Score:-56
Episode:2 Score:-58
Episode:3 Score:-60
Episode:4 Score:-60
Episode:5 Score:18


In [12]:
log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [13]:
model.learn(total_timesteps=40000)

Logging to Training/Logs/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -34.5    |
| time/              |          |
|    fps             | 833      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -38.2       |
| time/                   |             |
|    fps                  | 819         |
|    iterations           | 2           |
|    time_elapsed         | 4           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008520698 |
|    clip_fraction        | 0.06        |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 7.63e-05    |

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -25.7       |
| time/                   |             |
|    fps                  | 664         |
|    iterations           | 11          |
|    time_elapsed         | 33          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.013766906 |
|    clip_fraction        | 0.0679      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.09       |
|    explained_variance   | 0.000939    |
|    learning_rate        | 0.0003      |
|    loss                 | 34.7        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.00303    |
|    value_loss           | 68.4        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

In [15]:
shower_path = os.path.join('Training','Saved Models','Shower_Mode1_PPO')
model.save(shower_path)

In [16]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/vedant/miniconda3/envs/tensorflow/lib/python3.7/site-packages/stable_baselines3/common/evaluation.py:69: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  UserWarning,


(12.0, 58.787753826796276)